### Napari tests

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'

In [3]:
import napari

In [4]:
viewer = napari.Viewer()

2025-01-22 13:17:58.537 python[84009:7220973] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-22 13:17:58.537 python[84009:7220973] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Uing device: mps

Support for MPS devices is preliminary. SAM 2 is trained with CUDA and might give numerically different outputs and sometimes degraded performance on MPS. See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion.


Loaded SAM2VideoPredictor OCTRON
INFO: Hello and welcome to OCTRON!
Octopuses are amazing creatures 🐙


In [ ]:
current_indices = viewer.dims.current_step
current_indices

In [ ]:
viewer.dims.set_point(0,10)
current_indices = viewer.dims.current_step
print(current_indices)
image_data1 = viewer.layers[0].data[current_indices[0],:,:,:]
image_data1.shape
plt.imshow(image_data1)
plt.show() 
# Set to different index 
viewer.dims.set_point(0,11)
current_indices = viewer.dims.current_step
print(current_indices)
current_indices = viewer.dims.current_step
image_data2 = viewer.layers[0].data[current_indices[0],:,:,:]
image_data2.shape
plt.imshow(image_data2)


In [ ]:
(image_data1 == image_data2).all()